In [1]:
import argparse
import json
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import sys
from typing import List
from modeling_letter import LETTER
import torch
import transformers
import torch.nn as nn 
# from peft import PeftModel
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import LlamaForCausalLM, LlamaTokenizer, LlamaConfig, T5Tokenizer, T5Config, T5ForConditionalGeneration
import types
from utils import *
from collator import TestCollator, Collator
from evaluate import get_topk_results, get_metrics_results
from generation_trie import Trie

class SimpleArgs:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)
args = SimpleArgs(
    seed=42,
    gpu_id=0,
    tasks="seqrec",
    dataset="fashion",
    index_file=".index.epoch10000.alpha0-beta1e-4.json",
    data_path="/root/autodl-tmp/data",
    his_sep=", ",
    max_his_len=20,
    only_train_response=False,
    add_prefix=False,
    train_prompt_sample_num="1",
    train_data_sample_num="-1",
    valid_prompt_id=0,
    sample_valid=True,
    valid_prompt_sample_num=2,
    ckpt_path="./ckpt/Instruments/checkpoint-3465",
    results_file="./results/$DATASET/res.json",
    test_task="SeqRec",
    test_prompt_ids="0",
    sample_num=-1,
    num_beams=20,
    test_batch_size=20,
)
set_seed(args.seed)

device_map = {"": args.gpu_id}
device = torch.device("cuda",args.gpu_id)

config = T5Config.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained(
    "t5-small",
    model_max_length=512,
)
train_data, valid_data = load_datasets(args)
add_num = tokenizer.add_tokens(train_data.datasets[0].get_new_tokens())
config.vocab_size = len(tokenizer)

print("add {} new token.".format(add_num))
print("data num:", len(train_data))

# tokenizer = T5Tokenizer.from_pretrained(args.ckpt_path)
model = LETTER(config)
#model.set_hyper(args.temperature)
model.resize_token_embeddings(len(tokenizer))
model.to(device)
# model = T5ForConditionalGeneration.from_pretrained(
#     args.ckpt_path,
#     low_cpu_mem_usage=True,
#     device_map=device_map,
# )
prompt_ids = [0]

test_data = load_test_dataset(args)


# collator = TestCollator(args, tokenizer)
collator = Collator(args, tokenizer)
all_items = test_data.get_all_items()



candidate_trie = Trie(
        [
            [0] + tokenizer.encode(candidate)
            for candidate in all_items
        ]
    )
prefix_allowed_tokens = prefix_allowed_tokens_fn(candidate_trie)
train_loader = DataLoader(train_data,collate_fn=collator,batch_size=20)
# test_loader = DataLoader(test_data, batch_size=args.test_batch_size, collate_fn=collator,
                            # shuffle=True, num_workers=4, pin_memory=True)
for i in train_loader:
    break
input_ids = i['input_ids']
attention_mask = i['attention_mask']
actions = i['labels'].unsqueeze(1).repeat(1,2,1)

/root/miniconda3/envs/py310/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


add 2064 new token.
data num: 12547


In [9]:
res = model(input_ids=i['input_ids'].to(model.device),attention_mask=i['attention_mask'].to(model.device),labels=i['labels'].to(model.device))

In [11]:
res.decoder_hidden_states[0].shape

torch.Size([20, 6, 512])

In [14]:
res.decoder_hidden_state[0]

AttributeError: 'Seq2SeqLMOutput' object has no attribute 'decoder_hidden_state'